In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from torch.func import vmap, jacrev
from tqdm import tqdm
import os
import random
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import Callback
import math
from pydmd import DMD
from sklearn.preprocessing import MinMaxScaler
import warnings

In [2]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim, n_layers, dropout=0):
        super().__init__()
        layers = [nn.Linear(input_dim, hidden_dim), nn.ReLU()]
        for _ in range(n_layers):
            layers += [nn.Linear(hidden_dim, hidden_dim), nn.ReLU(), nn.Dropout(dropout)]
        layers += [nn.Linear(hidden_dim, output_dim)]
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

class AutoEncoder(nn.Module):
    def __init__(self, x_dim, u_dim, hidden_dim, n_layers, dropout=0):
        super().__init__()
        self.encoder = MLP(x_dim + u_dim, x_dim, hidden_dim, n_layers, dropout)
        self.decoder = MLP(x_dim + u_dim, x_dim, hidden_dim, n_layers, dropout)

    def forward(self, x, u=None, reverse=False):
        if u is not None:
            x = torch.cat([x, u], dim=-1)
        else:
            x = x
        if not reverse:
            return self.encoder(x)
        else:
            return self.decoder(x)
    
class CombinedNetwork(nn.Module):
    def __init__(self, ae, input_dim, lifted_dim, Xmax, Xmin):
        super(CombinedNetwork, self).__init__()
        self.input_dim = input_dim
        self.ae = ae  
        self.Xmax = Xmax
        self.Xmin = Xmin
        self.linear = nn.Linear(input_dim, lifted_dim, bias=False)  
        self._initialize_weights()
    
    def forward(self, x, u=None, reverse=False):
        x = x.float()
        Xmax = self.Xmax.to(x.device)
        Xmin = self.Xmin.to(x.device)
        if not reverse:
            x = (x - Xmin) / (Xmax - Xmin)
            chebyshev = torch.cos(self.linear(torch.arccos(x)))
            x = torch.cat((x, chebyshev), dim=-1)
            x = self.ae(x, u, reverse)
            return x
        else:
            x = self.ae(x, u, reverse)
            x = x[:, :self.input_dim]
            x = (Xmax - Xmin) * x + Xmin
            return x
    
    def _initialize_weights(self):
        lambda_s = 5
        self.linear.weight.data = torch.distributions.exponential.Exponential(lambda_s).sample(self.linear.weight.shape)

In [3]:
def dmd(model, X, U, rank):
    GX_pred_list = []
    GX_list = []
    U_list = []
    X_list = []
    GX = model(X, U.float())
    for i in range(X.shape[0]):
        GX_temp = GX[i, :, :].T
        dmd = DMD(svd_rank=rank, exact=True, sorted_eigs='abs')
        dmd.fit(GX_temp.cpu().detach().numpy())
        GX_pred = dmd.reconstructed_data.real
        GX_pred = np.array(GX_pred, dtype=np.float32)
        GX_pred = torch.from_numpy(GX_pred).cuda()
        GX_pred_list.append(GX_pred)
        GX_list.append(GX_temp)
        U_list.append(U[i, :, :].T)
        X_list.append(X[i, :, :].T)
    GX_pred = torch.cat(GX_pred_list, dim=-1)
    GX = torch.cat(GX_list, dim=1)
    U = torch.cat(U_list, dim=-1)
    X = torch.cat(X_list, dim=-1)

    return GX, GX_pred, U, X

In [4]:
class TrainModel(pl.LightningModule):
    def __init__(self, model, rank, learning_rate=1e-3, lamb=1, path="model_checkpoint_NP"):
        super(TrainModel, self).__init__()
        self.model = model
        self.learning_rate = learning_rate
        self.criterion = nn.MSELoss()
        self.best_val_loss = float('inf') 
        self.validation_outputs = []
        self.lamb = lamb
        self.train_losses = []
        self.rank = rank
        self.path = path+'.ckpt'

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        X_batch, U_batch = batch
        GY, GY_pred, U, Y = dmd(self.model, X_batch, U_batch, self.rank)
        Y_pred = self.model(GY_pred.T, U.T, reverse=True)
        loss_lin = self.criterion(GY, GY_pred)
        loss_recons = self.criterion(Y.T, Y_pred)

        loss = loss_lin + self.lamb * loss_recons
        self.log('train_loss', loss, on_step=True, on_epoch=False, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        Z_batch, U_batch = batch
        Z1, Z_pred, U, Z = dmd(self.model, Z_batch, U_batch, self.rank)
        Z_pred = self.model(Z_pred.T, U.T, reverse=True)
        valid_loss = self.criterion(Z_pred, Z.T)

        self.validation_outputs.append(valid_loss)
        self.log('val_loss', valid_loss)
        return valid_loss

    def test_step(self, batch, batch_idx):
        Z_batch, U_batch = batch
        Z1, Z_pred, U, Z = dmd(self.model, Z_batch, U_batch, self.rank)
        Z_pred = self.model(Z_pred.T, U.T, reverse=True)
        test_loss = self.criterion(Z_pred, Z.T)

        self.log('test_loss', test_loss)
        return test_loss
    
    def on_fit_start(self):
        if self.trainer.is_global_zero: 
            if os.path.exists("loss_log.txt"):
                os.remove("loss_log.txt")
            if os.path.exists(self.path):
                os.remove(self.path)
    
    def on_train_batch_end(self, outputs, batch, batch_idx):
        with torch.no_grad():  
            for name, module in self.model.named_modules():  
                if isinstance(module, nn.Linear): 
                    if name == "linear":  
                        continue
                    weight = module.weight  
                    sigma_max = torch.norm(weight, p=2)  
                    if sigma_max > 1:  
                        scale = (1 - 1e-3) / sigma_max
                        module.weight.data *= scale  
    
    def on_train_epoch_start(self):
        if os.path.exists(self.path):
            best_state_dict = torch.load(self.path)["state_dict"]
            self.load_state_dict(best_state_dict)
    
    def on_train_epoch_end(self):
        if self.trainer.is_global_zero: 
            avg_train_loss = self.trainer.callback_metrics.get("train_loss")
            if avg_train_loss is not None:
                self.train_losses.append(avg_train_loss.item())  
                print(f"Epoch {self.current_epoch}: Average Training Loss = {avg_train_loss.item()}")

    def on_validation_epoch_end(self):
        avg_val_loss = torch.stack(self.validation_outputs).mean()  
        self.log('avg_val_loss', avg_val_loss)
        self.validation_outputs.clear()
        print(f"Validation loss: {avg_val_loss}")
        with open("loss_log.txt", "a") as f:
            f.write(f"{avg_val_loss.item()}\n")

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate, eps=1e-08,
                                            weight_decay=0)
        scheduler = torch.optim.lr_scheduler.StepLR(
            optimizer,
            step_size=1,
            gamma=0.99
        )

        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss",  
            },
            "gradient_clip_val": 1.0,  
            "gradient_clip_algorithm": "norm",
        }

In [5]:
dim = 2  
hidden_dim = 27  
input_dim = 1
n_blocks = 3  
n_layers = 1
n_feature = 8
rank = 5
batch_size = 512
n_train = 10000
n_valid = 1000
n_test = 1000
dropout = 0
num_epochs = 100  
lamb = 1e-3
learning_rate = 1e-3  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
X_train = pd.read_csv('Non_X_train.csv', header=None).values
X_valid = pd.read_csv('Non_X_valid.csv', header=None).values
X_test = pd.read_csv('Non_X_test.csv', header=None).values
U_train = pd.read_csv('Non_U_train.csv', header=None).values
U_valid = pd.read_csv('Non_U_valid.csv', header=None).values
U_test = pd.read_csv('Non_U_test.csv', header=None).values

length = X_train.shape[1] // n_train
HX_train = []
HU_train = []
for i in range(n_train):
    HX_train.append(X_train[:, i*length:(i+1)*length])
    HU_train.append(U_train[:, i*length:(i+1)*length])
HX_train = np.stack([HX_train[idx].T for idx in range(n_train)], axis=0)
HU_train = np.stack([HU_train[idx].T for idx in range(n_train)], axis=0)
HX_valid = []
HU_valid = []
for i in range(n_valid):
    HX_valid.append(X_valid[:, i*length:(i+1)*length])
    HU_valid.append(U_valid[:, i*length:(i+1)*length])
HX_valid = np.stack([HX_valid[idx].T for idx in range(n_valid)], axis=0)
HU_valid = np.stack([HU_valid[idx].T for idx in range(n_valid)], axis=0)
train_dataset = TensorDataset(torch.tensor(HX_train, dtype=torch.float32), torch.tensor(HU_train, dtype=torch.float32))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
valid_dataset = TensorDataset(torch.tensor(HX_valid, dtype=torch.float32), torch.tensor(HU_valid, dtype=torch.float32))
valid_loader = DataLoader(valid_dataset, batch_size=99999, shuffle=True, num_workers=8, pin_memory=True)

X_result = np.concatenate([X_train, X_test, X_valid], axis=-1)
Xmax = torch.tensor(np.max(X_result, axis=-1), dtype=torch.float)
Xmin = torch.tensor(np.min(X_result, axis=-1), dtype=torch.float)

In [ ]:
warnings.filterwarnings("ignore")
path = "model_checkpoint_NP_ablate_AE"
checkpoint_callback = ModelCheckpoint(
    monitor="avg_val_loss",   
    dirpath="./ablation",  
    filename=path, 
    save_top_k=1,  
    mode="min",    
)
ae = AutoEncoder(x_dim=dim+n_feature, u_dim=input_dim, hidden_dim=hidden_dim, n_layers=n_layers, dropout=dropout)
model = CombinedNetwork(ae=ae, input_dim=dim, lifted_dim=n_feature, Xmax=Xmax, Xmin=Xmin)
lightning_model = TrainModel(model=model, rank=rank, learning_rate=learning_rate, lamb=lamb, path=path)
trainer = pl.Trainer(accelerator="gpu", devices=4, strategy="ddp_notebook", max_epochs=num_epochs, callbacks=[checkpoint_callback])

trainer.fit(lightning_model, train_loader, valid_loader)

In [7]:
ae = AutoEncoder(x_dim=dim+n_feature, u_dim=input_dim, hidden_dim=hidden_dim, n_layers=n_layers, dropout=dropout)
model = CombinedNetwork(ae=ae, input_dim=dim, lifted_dim=n_feature, Xmax=Xmax, Xmin=Xmin)
path = "model_checkpoint_NP_ablate_AE.ckpt"
lightning_model = TrainModel.load_from_checkpoint(path, model=model, rank=rank, learning_rate=learning_rate, map_location="cpu")
trainer = pl.Trainer(accelerator="gpu", devices=4, strategy="ddp_notebook", max_epochs=num_epochs)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [8]:
length = X_test.shape[1] // n_test
HX_test = []
HU_test = []
for i in range(n_test):
    HX_test.append(X_test[:, i*length:(i+1)*length])
    HU_test.append(U_test[:, i*length:(i+1)*length])
HX_test = np.stack([HX_test[idx].T for idx in range(n_test)], axis=0)
HU_test = np.stack([HU_test[idx].T for idx in range(n_test)], axis=0)
test_dataset = TensorDataset(torch.tensor(HX_test, dtype=torch.float32), torch.tensor(HU_test, dtype=torch.float32))
test_loader = DataLoader(test_dataset, batch_size=9999, shuffle=True)

In [9]:
warnings.filterwarnings("ignore")
trainer.test(lightning_model, dataloaders=test_loader)

You are using a CUDA device ('NVIDIA H100 80GB HBM3') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/4
You are using a CUDA device ('NVIDIA H100 80GB HBM3') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
You are using a CUDA device ('NVIDIA H100 80GB HBM3') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. 

Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  1.73it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.03086288832128048
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.03086288832128048}]

In [10]:
GY, GY_pred, U, _ = dmd(lightning_model.model, torch.tensor(HX_test, dtype=torch.float32), torch.tensor(HU_test, dtype=torch.float32), rank)
X_recons = lightning_model.model(GY_pred.cpu().T, U.T, reverse=True).T.detach().numpy()
X1 = X_recons.T.reshape(1000, -1)

In [13]:
pd.DataFrame((X1 - HX_test.reshape(1000, -1)) ** 2).to_csv('ae.csv', index=False)

In [11]:
np.mean((X1 - HX_test.reshape(1000, -1)) ** 2)

0.03196300690120273

In [ ]:
plt.plot(X_traj[0, :, 0], X_traj[0, :, 1], label='Test')
plt.plot(X_recons[0, :], X_recons[1, :], label='Reconstructed')
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.legend()
plt.title('Reconstructed Trajectory on Test Set')
# plt.savefig("controlled1.png", dpi=300, bbox_inches='tight')

In [ ]:
num_traj = 1000
dim = 2
rmse_values = np.zeros(61)
for i in range(num_traj):
    traj = HX_test[i, :61, :].reshape(1, 61, 2)
    GY, GY_pred, U, _ = dmd(lightning_model.model, torch.tensor(traj, dtype=torch.float32), torch.tensor(HU_test[i, :61, :], dtype=torch.float32).reshape(1, 61, 1), rank)
    X_recons = lightning_model.model(GY_pred.cpu().T, U.T, reverse=True).detach().numpy()
    error = np.sum((X_recons - traj.squeeze()) ** 2, axis=1)  
    rmse_values += error
rmse_values = np.sqrt(rmse_values / (num_traj * dim)) 

In [ ]:
t = np.linspace(0, 60*0.1, 61, traj.shape[1])
plt.semilogy(t, rmse_values)
plt.xlabel('$t$')
plt.ylabel('RMSE')
plt.title('Reconstructed Error on Test Set')
plt.savefig("controlled2.png", dpi=300, bbox_inches='tight')

In [ ]:
X_traj = torch.tensor(pd.read_csv('autonomous.csv', header=None).values.T, dtype=torch.float32).unsqueeze(0)
U_traj = torch.tensor(np.zeros(X_traj.squeeze(0).shape[:-1] + (1,)), dtype=torch.float32)
GX_temp, ldj = model(torch.tensor(X_traj.squeeze(0), dtype=torch.float32), U_traj)
GX_temp = GX_temp.T
dmd = DMD(svd_rank=rank, exact=True, sorted_eigs='abs')
dmd.fit(GX_temp.cpu().detach().numpy())
GX_pred = dmd.reconstructed_data.real
GX_pred = np.array(GX_pred, dtype=np.float32)
GX_pred = torch.from_numpy(GX_pred).cuda()
X_recons = lightning_model.model(GX_pred.cpu().T, U_traj, reverse=True).T.detach().numpy()

In [ ]:
plt.plot(X_traj[0, :, 0], X_traj[0, :, 1], label='Test')
plt.plot(X_recons[0, :], X_recons[1, :], label='Reconstructed')
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.legend()
plt.title('Reconstructed Trajectory Without Input')
plt.savefig("controlled3.png", dpi=300, bbox_inches='tight')